In [239]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [241]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [217]:
tweets["nretweets"].isnull().sum()

0

In [233]:
tweets["nretweets"].quantile([0.7,0.8,0.90,0.95,1])

0.70       49.00
0.80      104.00
0.90      311.00
0.95      871.00
1.00   541964.00
Name: nretweets, dtype: float64

In [240]:
def pipeline_qcut_label(dataset):
    dataset["categorias"] = pd.qcut(dataset["nretweets"], [0,0.8,0.90,0.95,1], labels=["very_unpopular", "unpopular","popular", "very_popular"])
    dataset["categorias"][dataset["categorias"].isnull()] = "very_unpopular"
    return pd.get_dummies(tweets, columns=["categorias"])

In [242]:
tweets_label_categorical = pipeline_qcut_label(tweets)

Establezco todos los tweets como poco populares de entrada, así si exiten nulos estos estarán como "poco populares". De esta forma, aquellos tweets que están por debajo o igual al percentil 80 serán como poco populares.

In [219]:
tweets["categorias"] = pd.qcut(tweets["nretweets"], [0,0.8,0.90,0.95,1], labels=["very_unpopular", "unpopular","popular", "very_popular"])

In [220]:
tweets["categorias"][tweets["categorias"].isnull()] = "very_unpopular"

In [221]:
tweets["categorias"].head(50)

0          unpopular
1            popular
2            popular
3            popular
4       very_popular
5            popular
6       very_popular
7     very_unpopular
8       very_popular
9          unpopular
10         unpopular
11           popular
12      very_popular
13      very_popular
14      very_popular
15         unpopular
16      very_popular
17      very_popular
18         unpopular
19      very_popular
20      very_popular
21      very_popular
22      very_popular
23      very_popular
24      very_popular
25      very_popular
26      very_popular
27      very_popular
28      very_popular
29      very_popular
30      very_popular
31      very_popular
32      very_popular
33    very_unpopular
34    very_unpopular
35         unpopular
36    very_unpopular
37    very_unpopular
38    very_unpopular
39    very_unpopular
40    very_unpopular
41    very_unpopular
42    very_unpopular
43    very_unpopular
44    very_unpopular
45    very_unpopular
46    very_unpopular
47    very_un

In [222]:
tweets[tweets["nretweets"]>871]["nretweets"].count()

2029

In [223]:
tweets[tweets["categorias"]=="very_popular"]["categorias"].count()

2029

In [224]:
tweets[tweets["nretweets"]<=104]["nretweets"].count()

32523

In [225]:
tweets[tweets["categorias"]=="very_unpopular"]["categorias"].count()

32523

In [226]:
tweets[tweets["categorias"].isnull()]["categorias"].count()

0

In [227]:
tweets_label_categorical = pd.get_dummies(tweets, columns=["categorias"])

In [243]:
tweets_label_categorical.head()

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,categorias_very_unpopular,categorias_unpopular,categorias_popular,categorias_very_popular
0,1425590913959612419,1425590913959612419,1628722334000.00,2021-08-12 00:52:14,200,NaN,RT @girlsalliance: We're so proud of the four ...,en,[],[],...,1425589589272678400.00,[],2021-08-12 00:46:58 CEST,NaN,NaN,NaN,0,1,0,0
1,1427736867739299841,1427736867739299841,1629233969000.00,2021-08-17 22:59:29,200,NaN,Some casual suggestions to 😏SLIDE😏 into when u...,en,"['shoesdaytuesday', 'afterskewlslide']",[],...,NaN,[],NaN,NaN,NaN,NaN,0,0,1,0
2,1427667300488937476,1427667300488937476,1629217383000.00,2021-08-17 18:23:03,200,NaN,RT @ValaAfshar: You are not your job.,en,[],[],...,1427647879246667776.00,[],2021-08-17 17:05:53 CEST,NaN,NaN,NaN,0,0,1,0
3,1427667012105371652,1427667012105371652,1629217315000.00,2021-08-17 18:21:55,200,NaN,What have we become 😔😂 Toddler Cites Freedom ...,en,[],[],...,NaN,[],NaN,NaN,NaN,NaN,0,0,1,0
4,1427497703596990467,1427497703596990467,1629176948000.00,2021-08-17 07:09:08,200,NaN,The tech giants that refuse to massively addre...,en,[],[],...,NaN,[],NaN,NaN,NaN,NaN,0,0,0,1


In [228]:
tweets_label_categorical["categorias_very_unpopular"]

0        0
1        0
2        0
3        0
4        0
        ..
40576    0
40577    1
40578    1
40579    0
40580    0
Name: categorias_very_unpopular, Length: 40581, dtype: uint8

In [245]:
tweets_label_categorical[tweets_label_categorical["categorias_very_unpopular"]<0]["categorias_very_unpopular"].count()

0

In [244]:
print(tweets_label_categorical[tweets_label_categorical["categorias_very_unpopular"].isnull()]["categorias_very_unpopular"].count())
print(tweets_label_categorical[tweets_label_categorical["categorias_unpopular"].isnull()]["categorias_very_unpopular"].count())
print(tweets_label_categorical[tweets_label_categorical["categorias_popular"].isnull()]["categorias_popular"].count())
print(tweets_label_categorical[tweets_label_categorical["categorias_very_popular"].isnull()]["categorias_very_popular"].count())

0
0
0
0
